In [1]:
# main.py
from main import gps_data_utils

import pandas as pd
from shapely.geometry import Point
# import geopandas as gpd
import folium
from joblib import Parallel, delayed

### Set Depot Points, Boundary and Buffer here

In [ ]:
p1 = Point(19.4230387681323, -99.1283375653607)
p2 = Point(19.4226931599795, -99.1282858845984)
p3 = Point(19.4223342584274, -99.128116747558)
p4 = Point(19.4215765747699, -99.1277173962126)
p5 = Point(19.4212024412726, -99.1275371303434)
p6 = Point(19.4204750900101, -99.1271367956332)
p7 = Point(19.4203918053066, -99.1270190501302)
p8 = Point(19.4174929683974, -99.1274678598945)
p9 = Point(19.4172999746193, -99.1275045888836)
p10 = Point(19.4176612061906, -99.1318657926695)
p11 = Point(19.4179003490392, -99.1319983339035)
p12 = Point(19.4237439344618, -99.1310950549938)
p13 = Point(19.4233598470446, -99.1283536711016)

points = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p1]
depot = gps_data_utils.set_depot_boundary(points,buffer = 0)

### Read and Validate GPS Data here

In [ ]:
'''
error_factor here is used as 1.5, i.e., 
    relative_error = abs(velocity_gps-velocity_vehicle) / velocity_vehicle 
    =>  ( 5 - 1 ) / 1 > 1.5
    => Calculated GPS speed is way higher than observed speed of the vehicle, i.e. gps walking
    => higher error_factor will allow more GPS walking
'''
error_factor = 1.25

In [ ]:
cols = ['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus']
date_col = ['RowReferenceTime']
d_format = 'ISO8601'
file = 'D:/Extra-Projects/MGL-India/Moving Dots/trip_data_10Units_1Year_V2.csv'

df = gps_data_utils.read_data(file,cols,date_col,d_format)
df = gps_data_utils.validate_mandatory_cols(df,'RowReferenceTime','lat','lon','Speed','UnityLicensePlate')
df = gps_data_utils.add_date_time_month_df(df)
df = df.groupby(by=['vehicleid','date']).apply(gps_data_utils.add_prev_latlong).reset_index(drop=True)
df = gps_data_utils.check_veh_within_depot(df,depot)

df['dist_prev_point'] = Parallel(n_jobs=-1, prefer="threads")(delayed(gps_data_utils.dist_calc_haversine_km)(row) for _, row in df.iterrows())

df['timediff'] = df.groupby(by=['vehicleid','date'])['timestamp'].diff().dt.total_seconds()/3600

df['velocity_gps'] = (df['dist_prev_point']/df['timediff']).fillna(0)
df['velocity_vehicle'] = (df['speed'] + df['speed'].shift(1)) / 2
df['speed_flag'] = df.apply(lambda row: ((row['velocity_vehicle'] != 0) and (abs((row['velocity_gps'] - row['velocity_vehicle']) / row['velocity_vehicle']) < error_factor)), axis=1)

In [ ]:
'''
Speed is now filtered to ease the GPS walking.
'''
df = df.loc[df['speed_flag']]

df= df.reset_index(drop=True)

In [ ]:
pd.DataFrame(df).to_csv('DF-speed-flagged.csv',index=False)

In [ ]:
df = pd.read_csv('DF-speed-flagged.csv',parse_dates=['timestamp','time'],date_format='ISO8601')
df = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
minlen_df = 10

maps = folium.Map()
mg = folium.FeatureGroup().add_to(maps)

# single_vehicle = None
single_vehicle = 'VIG21425'

if single_vehicle:
    tempdf = df[df['vehicleid'] == single_vehicle]

    # Save the interpolated data for each combination of 'vehicleid' and 'date'
    for name, group in tempdf.groupby(['vehicleid', 'date']):
        vehicleid,date = name
        
        if len(group) > minlen_df:
            group['timestamp'] = pd.to_datetime(group['timestamp'])
            group = group.set_index('timestamp')
            group = group.resample('1S').bfill().interpolate(method = 'linear')

            geometry = [Point(xy) for xy in zip(group['lon'].notna(),group['lat'].notna())]
            groupdf = gps_data_utils.df_to_gdf(group)
            
            
            file_name = f'vehicle_{vehicleid}_date_{date}_interpolated_data.csv'
            # group.to_csv(file_name, index=False)
            folium.PolyLine(list(zip(group['lat'],group['lon'])),color=gps_data_utils.rgb_to_hex(),weight=2).add_to(mg)

In [ ]:
# groupdf = group
groupdf = gps_data_utils.df_to_gdf(group,group['lat'],group['lon'])

In [ ]:
maps.fit_bounds(mg.get_bounds())
maps